# This graph learning treats every graph (CP) to be an episodic memory of the agent.

## Turn the raw data into RDF-data

I also did ontology engineering, which is saved at [`ontology-humemai.ttl`](./ontology-humemai.ttl)


In [1]:
import json

with open('./raw-data.json') as f:
    data = json.load(f)

In [2]:
import logging
from rdflib import URIRef, Namespace
from humemai import MemorySystem
from rdflib.namespace import RDF

from rdflib import RDF, XSD, BNode, Graph, Literal, Namespace, URIRef

# Turn off debug logging
logging.getLogger("humemai").setLevel(logging.WARNING)

humemai = Namespace("https://humem.ai/ontology#")
co_learning = Namespace("https://co_learning#")

memory_system = MemorySystem(verbose_repr=False)
memory_system.memory.graph.bind("co_learning", co_learning)


In [3]:
data_point = data[0]
data_point

{'cp_num': 0,
 'participant': 4069,
 'cp_name': 'Label',
 'ticks_lasted': 117,
 'round_num': 3,
 'timestamp': '2024-05-03T13:41:23',
 'unix_timestamp': 1714736483,
 'remaining_time': 2336,
 'remaining_rocks': 0,
 'victim_harm': 400,
 'situation': [[], []],
 'actionHuman': [[{'type': 'actor', 'content': 'Human'},
   {'type': 'action', 'content': 'Stand still in <location>'},
   {'type': 'location', 'content': '<Left> side of field'}],
  [],
  []],
 'actionRobot': [[{'type': 'action',
    'content': 'Break <object> in <location>'},
   {'type': 'object', 'content': 'Brown rock'}],
  [],
  []]}

In [4]:
event_properties = {
    co_learning.hasParticipantNumber: URIRef(
        co_learning + f"P{data_point['participant']}"
    ),
    co_learning.hasCPLabel: URIRef(co_learning + f"P{data_point['cp_name']}"),
    co_learning.hasTicksLasted: URIRef(co_learning + f"P{data_point['ticks_lasted']}"),
    co_learning.hasRoundNumber: URIRef(co_learning + f"P{data_point['round_num']}"),
    co_learning.hasTimeStammp: URIRef(co_learning + f"P{data_point['timestamp']}"),
    co_learning.hasUnixTimeStamp: URIRef(
        co_learning + f"P{data_point['unix_timestamp']}"
    ),
    co_learning.hasRemainingTime: URIRef(
        co_learning + f"P{data_point['remaining_time']}"
    ),
    co_learning.hasRemainingRocks: URIRef(
        co_learning + f"P{data_point['remaining_rocks']}"
    ),
    co_learning.hasVictimHarm: URIRef(co_learning + f"P{data_point['victim_harm']}"),
}

memory_system.memory.add_episodic_memory(
    triples=[(co_learning.USAR_bot, RDF.type, co_learning.Robot)],
    qualifiers={
        humemai.eventTime: Literal(data_point["timestamp"], datatype=XSD.dateTime),
        humemai.location: Literal("USAR"),
        humemai.event: co_learning.CP000,
    },
    event_properties=event_properties,
)

memory_system.memory.add_episodic_memory(
    triples=[
        (
            URIRef(co_learning + f"P{data_point['participant']}"),
            RDF.type,
            co_learning.Participant,
        )
    ],
    qualifiers={
        humemai.eventTime: Literal(data_point["timestamp"], datatype=XSD.dateTime),
        humemai.location: Literal("USAR"),
        humemai.event: co_learning.CP000,
    },
    event_properties=event_properties,
)

In [5]:
memory_system.memory.print_memories()

[USAR_bot, type, Robot, {'memoryID': '0', 'recalled': '0', 'eventTime': '2024-05-03T13:41:23', 'location': 'USAR', 'event': 'CP000'}]
[P4069, type, Participant, {'memoryID': '1', 'recalled': '0', 'eventTime': '2024-05-03T13:41:23', 'location': 'USAR', 'event': 'CP000'}]


In [6]:
memory_system.memory.print_events()

Event-related triples:
(CP000, hasParticipantNumber, P4069)
(CP000, hasCPLabel, PLabel)
(CP000, hasTicksLasted, P117)
(CP000, hasRoundNumber, P3)
(CP000, hasTimeStammp, P2024-05-03T13:41:23)
(CP000, hasUnixTimeStamp, P1714736483)
(CP000, hasRemainingTime, P2336)
(CP000, hasRemainingRocks, P0)
(CP000, hasVictimHarm, P400)
